In [1]:
#1) IMPORT LIBRARIES

#Computation and Structuring:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Modeling:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
#Testing:

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [4]:
#2) DATA IMPORT AND PRE-PROCESSING

#import full data set
df = pd.read_csv('Clean_Toronto_crime_analysis.csv') 

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


#import data set
df = pd.read_csv('Clean_Toronto_crime_analysis.csv') 

col_list = ['occurrenceyear','occurrencemonth','occurrenceday','occurrencedayofyear','occurrencedayofweek','occurrencehour','mci_category','Division',	'Hood_ID','premises_type']
df2 = df[col_list]
df2 = df2[df2['occurrenceyear'] > 2013]

#Factorize dependent variable column

crime_var = pd.factorize(df2['mci_category']) #codes the list of crimes to a int64 variable
df2['mci_category'] = crime_var[0]
definition_list_MCI = crime_var[1] #create an index reference 

#factorize premisetype:

premise_var = pd.factorize(df2['premises_type'])
df2['premises_type'] = premise_var[0]
definition_list_premise = premise_var[1] 

#factorize occurenceyear:

year_var = pd.factorize(df2['occurrenceyear'])
df2['occurrenceyear'] = year_var[0]
definition_list_year = year_var[1] 

#factorize occurencemonth:

month_var = pd.factorize(df2['occurrencemonth'])
df2['occurrencemonth'] = month_var[0]
definition_list_month = month_var[1] 

#factorize occurenceday:

day_var = pd.factorize(df2['occurrenceday'])
df2['occurenceday'] = day_var[0]
definition_list_day = day_var[1] 

#factorize occurencedayofweek:

dayweek_var = pd.factorize(df2['occurrencedayofweek'])
df2['occurrencedayofweek'] = dayweek_var[0]
definition_list_day = dayweek_var[1] 

#factorize division:

division_var = pd.factorize(df2['Division'])
df2['Division'] = division_var[0]
definition_list_division = division_var[1] 

#factorize HOOD_ID:

hood_var = pd.factorize(df2['Hood_ID'])
df2['Hood_ID'] = hood_var[0]
definition_list_hood = hood_var[1] 

#factorize occurencehour:

hour_var = pd.factorize(df2['occurrencehour'])
df2['occurrencehour'] = hour_var[0]
definition_list_hour = hour_var[1] 

#factorize occurencedayofyear:

dayyear_var = pd.factorize(df2['occurrencedayofyear'])
df2['occurrencedayofyear'] = dayyear_var[0]
definition_list_dayyear = dayyear_var[1] 

#set X and Y:

X = df2.drop(['mci_category'],axis=1).values #sets x and converts to an array
#print(X.head())
#print(X)
y = df2['mci_category'].values #sets y and converts to an array

#split the data into train and test sets for numeric encoded dataset:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

#need to OneHotEncode all the X variables for input into the classification model:

binary_encoder = OneHotEncoder(sparse=False)
encoded_X = binary_encoder.fit_transform(X)

X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size = 0.25, random_state = 21)


#--------------------------------------------------#

#3) MODELING AND TESTING:

#Numeric Encoded Model w/ SKLEARN:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) # Predicting the Test set results

print(accuracy_score(y_test, y_pred)) #accuracy at 0.63
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI)) 


#One Hot Encoded Model w/ SKLEARN:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
classifier.fit(X_train_OH, y_train_OH)
y_pred_OH = classifier.predict(X_test_OH) # Predicting the Test set results

print(accuracy_score(y_test_OH, y_pred_OH)) #modest improvement to 0.648
print(confusion_matrix(y_test_OH, y_pred_OH)) 
print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI)) #modest improvement

#Balanced Class Weight doesn't make a big difference for results:

classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42, class_weight='balanced')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test) 
print(accuracy_score(y_test, y_pred)) #accuracy at 0.63
print(confusion_matrix(y_test, y_pred)) 
print(classification_report(y_test,y_pred, target_names=definition_list_MCI)) 

#--------------------------------------------------#


grad_class = GradientBoostingClassifier(learning_rate=0.1,n_estimators = 10, random_state = 42)
grad_class.fit(X_train_OH, y_train_OH)
y_pred_OH = grad_class.predict(X_test_OH) # Predicting the Test set results

print(accuracy_score(y_test_OH, y_pred_OH)) #modest improvement to 0.648
print(confusion_matrix(y_test_OH, y_pred_OH)) 
print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI)) 

0.64636882077894
[[34058  2848    79   805  1859]
 [ 7221  6816   102    90   576]
 [ 1604   489    30    55   244]
 [ 3737   374    19  2452   522]
 [ 4330   877    38   299  4474]]
                 precision    recall  f1-score   support

        Assault       0.67      0.86      0.75     39649
Break and Enter       0.60      0.46      0.52     14805
     Theft Over       0.11      0.01      0.02      2422
        Robbery       0.66      0.35      0.45      7104
     Auto Theft       0.58      0.45      0.51     10018

       accuracy                           0.65     73998
      macro avg       0.52      0.42      0.45     73998
   weighted avg       0.62      0.65      0.62     73998

0.6571934376604773
[[34935  2503    51   577  1583]
 [ 7283  6846    81    54   541]
 [ 1673   455    26    23   245]
 [ 3927   373    13  2336   455]
 [ 4519   798    30   183  4488]]
                 precision    recall  f1-score   support

        Assault       0.67      0.88      0.76     39649
B

C:\Users\keyur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\keyur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\keyur\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
df.shape

(297353, 30)

In [29]:
df.head()

,Index_,event_unique_id,Division,occurrencedate,reporteddate,location_type,premises_type,ucr_code,ucr_ext,offence,...,occurrencedayofweek,occurrencehour,mci_category,Hood_ID,Neighbourhood,Longitude,Latitude,ObjectId,X,y
0,201,GO-20141273318,D31,2014/01/03 05:00:00+00,2014/01/03 05:00:00+00,"Apartment (Rooming House, Condo)",Apartment,1430,100,Assault,...,Friday,11.0,Assault,27,York University Heights,-79.504475,43.766371,1,-79.504475,43.766371
1,202,GO-20141274349,D42,2014/01/03 05:00:00+00,2014/01/03 05:00:00+00,"Single Home, House (Attach Garage, Cottage, Mo...",House,2120,200,B&E,...,Friday,14.0,Break and Enter,132,Malvern,-79.217083,43.814861,2,-79.217083,43.814861
2,203,GO-20141274052,D22,2014/01/03 05:00:00+00,2014/01/03 05:00:00+00,"Open Areas (Lakes, Parks, Rivers)",Outside,1430,100,Assault,...,Friday,13.0,Assault,19,Long Branch,-79.534957,43.589085,3,-79.534957,43.589085
3,204,GO-20141276966,D53,2014/01/03 05:00:00+00,2014/01/03 05:00:00+00,Other Commercial / Corporate Places (For Profi...,Commercial,2130,210,Theft Over,...,Friday,12.0,Theft Over,55,Thorncliffe Park,-79.346588,43.703212,4,-79.346588,43.703212
4,205,GO-20141274457,D22,2014/01/03 05:00:00+00,2014/01/03 05:00:00+00,Convenience Stores,Commercial,1610,210,Robbery - Business,...,Friday,14.0,Robbery,14,Islington-City Centre West,-79.531916,43.645247,5,-79.531916,43.645247


In [43]:
num_columns = df._get_numeric_data().columns
num_columns

Index(['Index_', 'ucr_code', 'ucr_ext', 'reportedyear', 'reportedday',
       'reporteddayofyear', 'reportedhour', 'occurrenceyear', 'occurrenceday',
       'occurrencedayofyear', 'occurrencehour', 'Longitude', 'Latitude',
       'ObjectId', 'X', 'y'],
      dtype='object')

In [44]:
cat_column = [col for col in df.columns if col not in num_columns]

cat_column 

['event_unique_id',
 'Division',
 'occurrencedate',
 'reporteddate',
 'location_type',
 'premises_type',
 'offence',
 'reportedmonth',
 'reporteddayofweek',
 'occurrencemonth',
 'occurrencedayofweek',
 'mci_category',
 'Hood_ID',
 'Neighbourhood']

In [84]:
X = final_df.drop(['offence','reportedhour','occurrenceyear','occurrencehour','occurrencedayofyear','occurrenceday'], axis = 1)
Y = final_df['offence']

In [85]:
X

,Index_,event_unique_id,Division,occurrencedate,reporteddate,location_type,premises_type,ucr_code,ucr_ext,reportedyear,...,occurrencemonth,occurrencedayofweek,mci_category,Hood_ID,Neighbourhood,Longitude,Latitude,ObjectId,X,y
0,201,1,1,1,1,1,1,1430,100,2014,...,1,1,1,1,1,-79.504475,43.766371,1,-79.504475,43.766371
1,202,2,2,1,1,2,2,2120,200,2014,...,1,1,2,2,2,-79.217083,43.814861,2,-79.217083,43.814861
2,203,3,3,1,1,3,3,1430,100,2014,...,1,1,1,3,3,-79.534957,43.589085,3,-79.534957,43.589085
3,204,4,4,1,1,4,4,2130,210,2014,...,1,1,3,4,4,-79.346588,43.703212,4,-79.346588,43.703212
4,205,5,3,1,1,5,4,1610,210,2014,...,1,1,4,5,5,-79.531916,43.645247,5,-79.531916,43.645247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297348,301112,259008,16,3542,3102,2,2,2135,210,2022,...,7,3,5,89,89,-79.308515,43.695118,301228,-79.308515,43.695118
297349,301113,259009,12,3542,3102,2,2,2135,210,2022,...,7,3,5,16,16,-79.455098,43.790363,301229,-79.455098,43.790363
297350,301114,259010,11,3542,3102,6,3,2135,210,2022,...,7,3,5,65,65,-79.234954,43.775740,301230,-79.234954,43.775740
297351,301115,259011,17,3508,3102,4,4,2135,210,2022,...,8,1,5,68,68,-79.281037,43.765801,301231,-79.281037,43.765801


In [86]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297353 entries, 0 to 297352
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Index_               297353 non-null  int64  
 1   event_unique_id      297353 non-null  int32  
 2   Division             297353 non-null  int32  
 3   occurrencedate       297353 non-null  int32  
 4   reporteddate         297353 non-null  int32  
 5   location_type        297353 non-null  int32  
 6   premises_type        297353 non-null  int32  
 7   ucr_code             297353 non-null  int64  
 8   ucr_ext              297353 non-null  int64  
 9   reportedyear         297353 non-null  int64  
 10  reportedmonth        297353 non-null  int32  
 11  reportedday          297353 non-null  int64  
 12  reporteddayofyear    297353 non-null  int64  
 13  reporteddayofweek    297353 non-null  int32  
 14  occurrencemonth      297353 non-null  int32  
 15  occurrencedayofwe

In [87]:
X.isnull().sum()

Index_                 0
event_unique_id        0
Division               0
occurrencedate         0
reporteddate           0
location_type          0
premises_type          0
ucr_code               0
ucr_ext                0
reportedyear           0
reportedmonth          0
reportedday            0
reporteddayofyear      0
reporteddayofweek      0
occurrencemonth        0
occurrencedayofweek    0
mci_category           0
Hood_ID                0
Neighbourhood          0
Longitude              0
Latitude               0
ObjectId               0
X                      0
y                      0
dtype: int64